## Archivo de ganancias de RTs para retención MX 

### Entradas 

1. Ganancias de los rappitenderos MX (storeekeeper_id, ganancia, propina, total)

2. Información de ciudades MX (storekeeper_id, identification, first_name, last_name, address, city)

3. Listado ciudades Guanajuato 

4. Base de datos de RFCs (storekeeper_id, first_name, rfc) 

5. Base de datos liliana ( nombre, storekeeper_id, rfc, penalizacion, facturado, error) 


### Proceso 

1. Leer las bases de datos de las entradas 

2. Crear un dataframe que se llame calculo_ret para incluir todos los calculos

3. Crear una columna con un valor bool para identificar cuales tienen RFC y cuales no. 

4. Traer el valor de la columna ganancias y propina del df de ganancias_mx para cada storekeeper_id

5. Crear columna base_iva (ganancia/1.16)

6. Crear columna total_servicio(base_iva + propina)

7. Crear columna iva (base_iva * 0.16)

8. Crear columna retención iva (condicional si tiene rfc entonces ret_iva = iva/2; si no tiene rfc ret_iva = iva/1)

9. Crear columna base_isr (propina + base_iva)

10. Crear columna retencion_isr (condicional si tiene rfc entonces ret_isr = 0.021 * base_isr; si no tiene rfc entonces ret_isr = 0.2 * base_isr)

11. Crear columna total_ret (ret_iva + ret_isr)

12. Traer información de las ciudades 

### salida 

1. Un archivo de excel que contenga tres hojas con la siguiente información: 

1.1. con rfc y sin rfc (storekeeper_id, estatus, rfc_query, rfc_liliana, penalizacion, monto_sin_propina, propinas, total_servicio, tasa, iva_trasladado(es igual a base_iva), iva_retenido, isr_retenido, monto_retenido, identificacion, first_name, last_name, address, city, guanajuato) 

1.2. Guanajuato (las mismas columnas que tengan true en Guanajuato) 

### 1. Importar librerias 

In [1]:
#Importar librerias 
import pandas as pd 
import numpy as np
import snowflake
import sqlalchemy
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

In [2]:
#Credenciales no ingresadas en repositorio publico por clausula de privacidad
snowflake_credentials = {'url' : ,
'account' : ,
'user':,
"authenticator": "externalbrowser",  
#"password":"0lXVwz8hV*vEA!",                            
'warehouse': ,
'database' : 
}

In [3]:
engine = sqlalchemy.create_engine(URL(**snowflake_credentials), encoding = 'latin-1')
connection = engine.connect()

### 2. Definición de las fechas 

In [4]:
fecha_inicio = input('fecha_inicio: ')
fecha_final = input('fecha_final: ')

fecha_inicio: 2021-09-01
fecha_final: 2021-09-30


### 3. Bases de datos

#### 3.1. Ganancias de los rappitenderos durante el periodo seleccionado

In [5]:
warehouse = pd.read_sql_query("USE NOMBRE_WAREHOUSE",connection)
ganancias_rts = pd.read_sql_query(
f'''
QUERY
''', connection
)

In [7]:
warehouse = pd.read_sql_query("USE NOMBRE_WAREHOUSE",connection)
propina_rts = pd.read_sql_query(
f'''
QUERY
''',connection
)

In [9]:
ganancias_propinas = ganancias_rts.merge(propina_rts, how = 'left',left_on='storekeeper_id',right_on='skt_id')

In [10]:
ganancias_propinas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65703 entries, 0 to 65702
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   storekeeper_id  65703 non-null  int64  
 1   rt              65703 non-null  bool   
 2   ganancia        65703 non-null  float64
 3   skt_id          65035 non-null  float64
 4   propina         65035 non-null  float64
dtypes: bool(1), float64(3), int64(1)
memory usage: 2.6 MB


#### 3.2. Información de las ciudades 

In [11]:
warehouse = pd.read_sql_query("USE NOMBRE_WAREHOUSE",connection)
info_ciudades = pd.read_sql_query(
f'''
QUERY
''', connection

)

#### 3.3. Base de datos de rfcs

In [13]:
warehouse = pd.read_sql_query("USE WAREHOUSE PAGOS_ANALYSTS",connection)
bd_rfcs = pd.read_sql_query(
'''
query
''', connection
)

#### 3.4. Listado de ciudades de Guanajuato 

In [15]:
guanajuato = pd.read_csv('nombre.csv',encoding='UTF-8')

### 4. Calculo de los valores

#### 4.1. Validacion de rts con RFC

In [17]:
calculos_ret = ganancias_propinas[['storekeeper_id','ganancia','propina']]

In [18]:
calculos_ret.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65703 entries, 0 to 65702
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   storekeeper_id  65703 non-null  int64  
 1   ganancia        65703 non-null  float64
 2   propina         65035 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 2.0 MB


In [19]:
calculos_ret_rfc = calculos_ret.merge(bd_rfcs,how='left',left_on='storekeeper_id',right_on='storekeeper_id')   

In [20]:
calculos_ret_rfc['rfc'] = calculos_ret_rfc['rfc'].fillna('sin rfc')

In [21]:
calculos_ret_rfc['propina'] = calculos_ret_rfc['propina'].fillna(0)

In [23]:
for i in calculos_ret_rfc.index:
    if calculos_ret_rfc.loc[i,'rfc'] == 'sin rfc': 
        calculos_ret_rfc.loc[i,'validacion_rfc'] = False
    else:
        calculos_ret_rfc.loc[i,'validacion_rfc'] = True

In [24]:
calculos_ret_rfc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65703 entries, 0 to 65702
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   storekeeper_id  65703 non-null  int64  
 1   ganancia        65703 non-null  float64
 2   propina         65703 non-null  float64
 3   first_name      46658 non-null  object 
 4   rfc             65703 non-null  object 
 5   validacion_rfc  65703 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 5.5+ MB


#### 4.2. Calculo de los valores de retención

In [25]:
for i in calculos_ret_rfc.index:
    calculos_ret_rfc.loc[i,'base_iva'] = calculos_ret_rfc.loc[i,'ganancia']/1.16
    calculos_ret_rfc.loc[i,'total_servicio'] = calculos_ret_rfc.loc[i,'base_iva'] + calculos_ret_rfc.loc[i,'propina']
    calculos_ret_rfc.loc[i,'iva'] = calculos_ret_rfc.loc[i,'base_iva']*0.16
    if calculos_ret_rfc.loc[i,'validacion_rfc'] == True:
        calculos_ret_rfc.loc[i,'ret_iva'] = calculos_ret_rfc.loc[i,'iva']/2
        calculos_ret_rfc.loc[i,'ret_isr'] = calculos_ret_rfc.loc[i,'total_servicio']*0.021
    else: 
        calculos_ret_rfc.loc[i,'ret_iva'] = calculos_ret_rfc.loc[i,'iva']/1
        calculos_ret_rfc.loc[i,'ret_isr'] = calculos_ret_rfc.loc[i,'total_servicio']*0.2
          
    calculos_ret_rfc.loc[i,'total_ret'] = calculos_ret_rfc.loc[i,'ret_iva'] + calculos_ret_rfc.loc[i,'ret_isr']     
    
    

In [26]:
calculos_ret_rfc = calculos_ret_rfc.round(2)

In [27]:
calculos_ret_rfc

,storekeeper_id,ganancia,propina,first_name,rfc,validacion_rfc,base_iva,total_servicio,iva,ret_iva,ret_isr,total_ret
0,474053,49415.30,6694.0,Mario Alberto,EARM890414PA5,True,42599.40,49293.40,6815.90,3407.95,1035.16,4443.11
1,244232,46215.70,2812.0,Jose Rene,JILR820702646,True,39841.12,42653.12,6374.58,3187.29,895.72,4083.01
2,354027,44308.40,7827.0,Norberto Daniel,TARN8912114J6,True,38196.90,46023.90,6111.50,3055.75,966.50,4022.25
3,219540,41884.55,11410.0,José Francisco,MAMF641004IL9,True,36107.37,47517.37,5777.18,2888.59,997.86,3886.45
4,248628,40867.45,4932.0,Christopher Daniel,MOMC920828UTA,True,35230.56,40162.56,5636.89,2818.44,843.41,3661.86
...,...,...,...,...,...,...,...,...,...,...,...,...
65698,630463,0.00,0.0,NaN,sin rfc,False,0.00,0.00,0.00,0.00,0.00,0.00
65699,564495,0.00,6442.0,NaN,sin rfc,False,0.00,6442.00,0.00,0.00,1288.40,1288.40
65700,56650,0.00,5082.0,NaN,sin rfc,False,0.00,5082.00,0.00,0.00,1016.40,1016.40
65701,644476,-35.00,1265.0,NaN,sin rfc,False,-30.17,1234.83,-4.83,-4.83,246.97,242.14


In [28]:
tabla_final = calculos_ret_rfc.merge(info_ciudades,left_on='storekeeper_id',right_on='storekeeper_id')

In [29]:
tabla_final

,storekeeper_id,ganancia,propina,first_name_x,rfc,validacion_rfc,base_iva,total_servicio,iva,ret_iva,ret_isr,total_ret,identification,first_name_y,last_name,address,city
0,474053,49415.30,6694.0,Mario Alberto,EARM890414PA5,True,42599.40,49293.40,6815.90,3407.95,1035.16,4443.11,EARM890414HBCSDR05,Mario Alberto,Estrada Rodríguez,calle marmoleros sur #1475 Colonia industrial,Mexicali
1,244232,46215.70,2812.0,Jose Rene,JILR820702646,True,39841.12,42653.12,6374.58,3187.29,895.72,4083.01,JILR820702HNLMXN05,Jose Rene,Jimenez Loa,sierra de mico 1107,Monterrey
2,354027,44308.40,7827.0,Norberto Daniel,TARN8912114J6,True,38196.90,46023.90,6111.50,3055.75,966.50,4022.25,TARN891211HDFPMR06,Norberto Daniel,Tapia Romero,1er callejón de gral a aya #69 int6,Tijuana
3,219540,41884.55,11410.0,José Francisco,MAMF641004IL9,True,36107.37,47517.37,5777.18,2888.59,997.86,3886.45,MAMF641004HSPRNR02,José Francisco,Martínez Mendoza,priv mirador 3549 col cerro de la campana 64760,Monterrey
4,248628,40867.45,4932.0,Christopher Daniel,MOMC920828UTA,True,35230.56,40162.56,5636.89,2818.44,843.41,3661.86,MOMC920828HNLNRH04,Christopher Daniel,Montalvo Martinez,Re. San Felipe #230. san nicolas de los garza,Monterrey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65654,597222,0.00,10.0,NaN,sin rfc,False,0.00,10.00,0.00,0.00,2.00,2.00,HEGK011114MGTRRRA4,Karla Janeth,Hernandez Garcia,Zempoala 60559,León
65655,645121,0.00,20.0,NaN,sin rfc,False,0.00,20.00,0.00,0.00,4.00,4.00,GAPM781222MTSRRR06,Maira Yolanda,Garcia Perez,Real de Santa Catalina 123 Real de San José.Ju...,Monterrey
65656,600282,0.00,20.0,NaN,sin rfc,False,0.00,20.00,0.00,0.00,4.00,4.00,GAVE950824HVZRLR01,ERICK MOISES,GARCIA VELAZQUEZ,Xóchitl fama ll santa catarina nl.,Monterrey
65657,650537,0.00,20.0,NaN,sin rfc,False,0.00,20.00,0.00,0.00,4.00,4.00,LOSJ980208HDFPYL04,Joel Esteban,Lopez Sayon,c Jilotepec mz 3 lt 6 col Chimalhuacán edo Méx,Ciudad de México


In [30]:
list(guanajuato['ciudades'])

['Abasolo',
 'Acámbaro',
 'Apaseo el Alto',
 'Apaseo el Grande',
 'Atarjea',
 'Celaya',
 'Comonfort',
 'Coroneo',
 'Cortazar',
 'Cuerámaro',
 'Doctor Mora',
 'Dolores Hidalgo C.I.N.',
 'Guanajuato',
 'Huanímaro',
 'Irapuato',
 'Jaral del Progreso',
 'Jerécuaro',
 'León',
 'Manuel Doblado',
 'Moroleón',
 'Ocampo',
 'Pénjamo',
 'Pueblo Nuevo',
 'Purísima del Rincón ',
 'Purísima del Rincón',
 'Romita',
 'Salamanca',
 'Salvatierra',
 'San Diego de la Unión',
 'San Felipe',
 'San Francisco del Rincón',
 'San José Iturbide',
 'San Luis de la Paz',
 'San Miguel de Allende',
 'Santa Catarina',
 'Santa Cruz de Juventino Rosas',
 'Juventino Rosas',
 'Santiago Maravatío',
 'Silao',
 'Tarandacuao',
 'Tarimoro',
 'Tierra Blanca',
 'Uriangato',
 'Valle de Santiago',
 'Victoria',
 'Villagrán',
 'Xichú',
 'Yuriria']

In [31]:
for i in tabla_final.index:
    if tabla_final.loc[i,'city'] in list(guanajuato['ciudades']):
        tabla_final.loc[i,'guanajuato'] = True
    else:
        tabla_final.loc[i,'guanajuato'] = False
        

In [32]:
tabla_guanajuato = tabla_final[tabla_final['guanajuato']== True]

In [33]:
mes = input('mes: ')

mes: Septiembre


In [34]:
with pd.ExcelWriter(f'C:/ruta/archivos_automatizacionretenciones_rts_{mes}.xlsx') as writer:  
    tabla_final.to_excel(writer, sheet_name='Consolidado')
    tabla_guanajuato.to_excel(writer, sheet_name='GUANAJUATO')